In [1]:
from qiskit_metal import designs, MetalGUI
from qiskit_metal.qlibrary.sample_shapes.rectangle import Rectangle
from qiskit_metal.qlibrary.sample_shapes.circle_raster import CircleRaster

In [2]:
# all dims in um
via_rad = 10
via_base_rad = via_rad+5
stub_width = 10
stub_length = 250
stub_subs_margin = 6

In [3]:
design = designs.MultiPlanar({}, overwrite_enabled=True, layer_stack_filename="via_layer_stack.txt")
gui = MetalGUI(design)

In [4]:
design.chips.main.size.size_x = '1mm'
design.chips.main.size.size_y = '1mm'
design.variables.sample_holder_top = '320um'
design.variables.sample_holder_bottom = '200um'

In [5]:
design.ls.ls_df

,chip_name,layer,datatype,material,thickness,z_coord,fill
0,'main',1,0,'pec','5um','0um','True'
1,'main',2,0,'pec','10um','5um','True'
2,'main',3,0,'pec','5um','15um','True'
3,'main',4,0,'silicon','-200um','0um','True'


In [6]:
design.delete_all_components()

In [7]:
# Layer1 definitions


# stub1 metal options
stub1_met_ops = { 'width': str(stub_length-via_base_rad)+'um',
                  'height': str(stub_width)+'um',
                  'pos_x': str(-stub_length/2-stub_length/2-via_base_rad/2)+'um',
                  'pos_y': '0',
                  'orientation': '0',
                  'subtract': 'False',
                  'helper': 'False',
                  'chip': 'main',
                  'layer': '1' }

stub1_subs_ops = { 'width': str(stub_length-via_base_rad)+'um',
                  'height': str(stub_width+stub_subs_margin)+'um',
                  'pos_x': str(-stub_length/2-stub_length/2-via_base_rad/2)+'um',
                  'pos_y': '0',
                  'orientation': '0',
                  'subtract': 'True',
                  'helper': 'False',
                  'chip': 'main',
                  'layer': '1' }

# bottom via metal options
via_bot_met_ops = { 'radius': str(via_base_rad)+'um',
                    'resolution': '16',
                    'pos_x': str(-stub_length/2-via_base_rad/2)+'um',
                    'pos_y': '0',
                    'subtract': 'False',
                    'helper': 'False',
                    'chip': 'main',
                    'layer': '1' }

# bottom via subs options
via_bot_subs_ops = { 'radius': str(via_base_rad+stub_subs_margin)+'um',
                    'resolution': '16',
                    'pos_x': str(-stub_length/2-via_base_rad/2)+'um',
                    'pos_y': '0',
                    'subtract': 'True',
                    'helper': 'False',
                    'chip': 'main',
                    'layer': '1' }

Rectangle(design, 'stub1_met', stub1_met_ops)
Rectangle(design, 'stub1_subs', stub1_subs_ops)
CircleRaster(design, 'via_bot_met', via_bot_met_ops)
CircleRaster(design, 'via_bot_subs', via_bot_subs_ops)
gui.rebuild()

In [8]:
# Layer2 definitions

# via metal options
via_met_ops = { 'radius': str(via_rad)+'um',
                'resolution': '16',
                'pos_x': str(-stub_length/2-via_base_rad/2)+'um',
                'pos_y': '0',
                'subtract': 'False',
                'helper': 'False',
                'chip': 'main',
                'layer': '2' }

CircleRaster(design, 'via_met', via_met_ops)
gui.rebuild()

### GMSH RENDERER

In [9]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer


gmsh = QGmshRenderer(design, layer_types=dict(metal=[1,2,3], dielectric=[4]))

gmsh.options.mesh.min_size = '5um'
gmsh.options.mesh.max_size = '50um'

In [10]:
gmsh.render_design(mesh_geoms=False, skip_junctions=True)
#gmsh.add_mesh(dim=3, intelli_mesh=False)
#gmsh.launch_gui()
#gmsh.close()

Exception: Intersection operation cannot be performed